In [60]:
import numpy as np
import math
from scipy.stats import poisson, norm, bernoulli, gamma
from scipy.special import psi

#nneuron = 41
#sprefs = np.linspace(-60, 60, nneuron)

nneuron = 5
sprefs = np.linspace(-20, 20, nneuron)

sig0_sq      = float(3**2)
sig1_sq      = float(12**2)
sigtc_sq     = float(10**2)
ndatapergain = 3000
mu = 0
gain = 15

def generate_popcode_data(ndatapergain, nneuron, sig0_sq, sig1_sq, sigtc_sq, gain, noise):
    s = np.random.normal(mu, np.sqrt(sig0_sq), ndatapergain)
    sprefs_data = np.tile(sprefs, (ndatapergain, 1))
    r = gain * np.exp(-np.square((np.transpose(np.tile(s, (nneuron, 1))) - sprefs_data))/(2 * sigtc_sq))
    if noise == "poisson":
        r = np.random.poisson(r) + 0.0
    return r, s

In [83]:
#simple version
r, actual = generate_popcode_data(ndatapergain, nneuron, sig0_sq, sig1_sq, sigtc_sq, 1, "poisson")
def norm_g(zTw):
    return zTw + 3

def relu(zTw):
    return max(0.000000000000001, zTw)

def g_norm(z, loc, scale):
    d_loc = (z - loc)/scale
    d_scale = -(1/(2 * scale)) + ((z - loc)**2)/(2 * scale**2)
    return [d_loc, d_scale]

L = 1
s = 1
z = [None]*L
W = [None]*L
p_z = [None]*(L + 1)
q_z = [None]*L
p_W = [None]*L
q_W = [None]*L
g = [None]*L
g_W = [None]*L

xi_normal = np.random.rand(nneuron, 2) #[mu, sigma]
train_data = r[1:3000]
actual = actual[1:3000]
rho = .001
rhoW = .01

num = 1
for i in range(len(train_data)):
    n = train_data[i]
    true = actual[i]
    converged = False
    lam_normal = np.random.rand(1, 2)[0] * 3 #[mu, sigma]
    while not converged:
        #normal
        z_loc, z_scale = lam_normal
        z[0] = np.random.normal(loc=z_loc, scale=z_scale, size=s)
        W[0] = [None]*len(xi_normal)
        for i in range(len(xi_normal)):
            W_loc, W_scale = xi_normal[i]
            W[0][i] = np.random.normal(loc=W_loc, scale=W_scale, size=s)
        np.concatenate(W[0])
        p_z[0] = [None] * len(W[0])
        q_z[0] = [None] * len(z[0])
        g[0] = [None] * len(z[0])
        g_W[0] = [None] * len(W[0])
        params = z[0] * W[0]
        for i in range(len(n)):
            lam_p = relu(params[i][0])
            p_z[0][i] = poisson.pmf(n[i], lam_p)
            g_W[0][i] = g_norm(W[0][i], W_loc, W_scale)
        np.concatenate(g_W[0])
        for i in range(len(z[0])):
            q_z[0][i] = norm.pdf(x=z[0][i], loc=z_loc, scale=z_scale)
            g[0][i] = g_norm(z[0][i], z_loc, z_scale)
        p_z[0] = np.log(p_z[0])
        q_z[0] = np.log(q_z[0])
        
        p_z[1] = [norm.pdf(x=z[0][i], loc=mu, scale=np.sqrt(sig0_sq))]
        
        l = 0
        S = [None] * L
        T = [None] * L
        for l in range(L):
            S[l] = [None] * len(z[l])
            T[l] = [None] * len(W[l])
            pzl = np.sum(p_z[l])
            for k in range(len(z[l])):
                glk = np.asarray(g[l][k])
                S[l][k] = glk * (pzl + p_z[l+1][k] + q_z[l][k]) #???
            for j in range(len(W[l])):
                gWlj = np.asarray(g_W[l][j])
                #T[l][j] = gWlj * pzl #???
                T[l][j] = gWlj * p_z[l][j] #???
                np.concatenate(T[l][j])
        lam_normal += list(rho * S[0][0])
        lam_normal[1] = relu(lam_normal[1])
        for m in range(len(xi_normal)):
            rT0n = rhoW * T[0][m]
            rT0n = [rT0n[0][0], rT0n[1][0]]
            xi_normal[m] += rT0n
            xi_normal[m][1] = relu(xi_normal[m][1])
        num += 1
        if num > 1000:
            converged = True
            print "x", n
            print "lambda", lam_normal
            print "true", true
            print "xi", xi_normal
            print
            num = 1
            rhoW = rhoW * .9
            rho = .001

x [ 0.  1.  0.  1.  0.]
lambda [ 4.4468176   0.96826109]
true 3.6178039887
xi [[ -8.41225409e-01   2.68385485e-01]
 [  1.82639926e-01   1.00000000e-15]
 [ -7.45742623e-01   2.77516040e-01]
 [ -5.38716760e-01   7.76626143e-01]
 [ -8.65816188e-01   3.39627170e-01]]

x [ 0.  1.  2.  1.  0.]
lambda [-2.23759115  0.99078991]
true -1.37957157311
xi [[  4.37331548e-01   4.47144831e-01]
 [ -3.48721647e-01   1.66743419e-01]
 [ -1.57237555e+00   1.00000000e-15]
 [ -6.96592094e-01   1.00000000e-15]
 [  4.87676195e-01   4.49119027e-01]]

x [ 0.  2.  2.  0.  0.]
lambda [ 1.43281712  0.82177885]
true -5.39907363469
xi [[ -5.72254872e-01   2.38751627e-01]
 [  2.77537539e-01   1.00000000e-15]
 [  7.08335106e-01   1.00000000e-15]
 [ -5.65295301e-01   5.72488309e-02]
 [ -6.67201259e-01   3.10860663e-01]]

x [ 0.  0.  2.  0.  0.]
lambda [ 1.67269196  0.50266541]
true -3.26135778886
xi [[ -6.24978985e-01   3.40196785e-01]
 [  1.89777530e-12   1.00000000e-15]
 [ -5.04619967e-01   2.55253503e-01]
 [ -5.5511

/Users/ocnc2015/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:59: RuntimeWarning: divide by zero encountered in log
/Users/ocnc2015/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:10: RuntimeWarning: invalid value encountered in double_scalars
/Users/ocnc2015/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:11: RuntimeWarning: invalid value encountered in double_scalars


KeyboardInterrupt: 